# Imports

In [77]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [3]:
df = pd.read_csv('../Data/merged_vehicle_data.csv')
df.head()

,make,model,engine_size,trim,num_doors,mileage,reg_year,transmission_type,fuel_type,price,is_electric
0,BMW,1 Series,1.5,116d Sport,2,65221,2016,Manual,Diesel,9400,0
1,BMW,1 Series,2.0,120d M Sport,2,77873,2015,Manual,Diesel,9950,0
2,BMW,1 Series,1.5,118i Sport,2,57371,2016,Manual,Petrol,10050,0
3,BMW,1 Series,2.0,118d M Sport,2,71342,2016,Manual,Diesel,10450,0
4,BMW,1 Series,1.5,118i M Sport,2,77767,2017,Manual,Petrol,11000,0


# Model 1

In [6]:
cvec1 = CountVectorizer()
trimcvec = cvec1.fit_transform(df['trim'])
model1 = pd.DataFrame(trimcvec.todense(), columns=cvec1.get_feature_names_out())
model1[['model','make','engine_size', 'num_doors', 'mileage', 'reg_year', 'transmission_type',
        'fuel_type','is_electric', 'price']] = df[['model', 'make','engine_size', 'num_doors',
            'mileage','reg_year','transmission_type','fuel_type','is_electric', 'price']].copy()

In [7]:
model1 = pd.get_dummies(model1, columns=['make','model','transmission_type',
                                         'fuel_type'], drop_first=True)

In [8]:
X = model1.drop(columns=['price'])
y = model1['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [37]:
extra = ExtraTreesRegressor(n_estimators=500)
extra.fit(X_train, y_train)
extra.score(X_train, y_train), extra.score(X_test, y_test)

(0.9999994242435667, 0.9923324905845222)

In [38]:
extra_pred = extra.predict(X_test)
metrics.mean_absolute_error(y_test, extra_pred)

135.90025083307913

# Model 2

In [20]:
params = {
    'max_features': np.arange(1, X.shape[1] + 1), # p
    'max_depth': np.append(np.arange(1, 10), None), # 10
    'min_samples_leaf': np.arange(1, 31) # 30
}

rf = RandomForestRegressor(
    n_estimators=100,
    random_state=2023
)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
rs = RandomizedSearchCV(rf, params, n_iter=100, cv=kf, n_jobs=-1);

In [21]:
%%time
rs.fit(X_train, y_train)

CPU times: user 18.3 s, sys: 1.31 s, total: 19.7 s
Wall time: 23min 18s


RandomizedSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=RandomForestRegressor(random_state=2023),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, None], dtype=object),
                                        'max_features': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,...
       911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923,
       924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936,
       937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949,
       950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962,
       963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975,
       976, 977, 978, 979]),
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])})

In [22]:
rs.score(X_train, y_train), rs.score(X_test, y_test)

(0.984864078026492, 0.9794510227891288)

In [23]:
rs_preds = rs.predict(X_test)
metrics.mean_absolute_error(y_test, rs_preds)

463.50345814754127

# Model 3

In [51]:
import dask_ml.model_selection as dcv
from sklearn.ensemble import ExtraTreesRegressor
import dask.array as da


In [54]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [2, 3],
    'max_depth': [None, 10, 20, 30, 40, 50]
}

et = ExtraTreesRegressor(n_estimators=500, random_state=42)

et = dcv.RandomizedSearchCV(et, params, n_iter=100, cv=kf, n_jobs=-1)

In [55]:
%%time
et.fit(X_train, y_train)
et.score(X_train, y_train), et.score(X_test, y_test)

KeyboardInterrupt: 

# Model 4

In [83]:
pgrid = {
    'learning_rate': [0.1, 1],
    'n_estimators': [10, 100, 200, 300],
    'max_depth': [None, 1, 2, 3]
}
grad = GradientBoostingRegressor()
gs = GridSearchCV(grad, pgrid, cv=kf, n_jobs=-1)

In [84]:
%%time
gs.fit(X_train, y_train)
gs.score(X_train, y_train), gs.score(X_test, y_test)

CPU times: user 1min 14s, sys: 664 ms, total: 1min 15s
Wall time: 21min 8s


(0.9999995720026177, 0.9863771406562538)

In [85]:
gs_preds = gs.predict(X_test)
metrics.mean_absolute_error(y_test, gs_preds)

196.8277119001801

# Model 5

In [86]:
ss = StandardScaler()
X_trainss = ss.fit_transform(X_train)
X_testss = ss.transform(X_test)

In [87]:
et2 = ExtraTreesRegressor(n_estimators=500)
et2.fit(X_trainss, y_train)
et2.score(X_trainss, y_train), et2.score(X_testss, y_test)

(0.9999994092328096, 0.9923559639833011)

In [90]:
et2_preds = et2.predict(X_testss)
metrics.mean_absolute_error(y_test, et2_preds)

135.8060487851758

# Model 6

In [91]:
%%time
tree = DecisionTreeRegressor()
tree.fit(X_trainss, y_train)
tree.score(X_trainss, y_train), tree.score(X_testss, y_test)

CPU times: user 406 ms, sys: 26.6 ms, total: 433 ms
Wall time: 441 ms


(0.9999995720026178, 0.9857788454091585)

In [92]:
tree_preds = tree.predict(X_testss)
metrics.mean_absolute_error(y_test, tree_preds)

204.8193318183469

# Model 7

In [96]:
%%time
ada = AdaBoostRegressor()
ada.fit(X_train, y_train)
ada.score(X_train, y_train), ada.score(X_test, y_test)

CPU times: user 17.2 s, sys: 1.68 s, total: 18.9 s
Wall time: 19.1 s


(0.5506858344942585, 0.56608896458223)

In [97]:
ada_preds = ada.predict(X_testss)
metrics.mean_absolute_error(y_test, ada_preds)

/Users/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(


4769.18844867847

# Model 8

In [99]:
pgrid = {
    'base_estimator__max_depth': [None, 1, 2, 3, 4, 5, 6, 7],
    'base_estimator__min_samples_leaf': np.arange(11, 22, 2) 
}

tree = DecisionTreeRegressor()
bag = BaggingRegressor(tree, n_estimators = 500, random_state=42)
gs = GridSearchCV(bag, pgrid, cv=kf, n_jobs=-1)

In [100]:
%%time
gs.fit(X_train, y_train)

/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: P

/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: P

/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: P

/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: P

/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: P

...

/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: P

/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: P

/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
/Users/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:720: FutureWarning: P

CPU times: user 1min 48s, sys: 4.32 s, total: 1min 52s
Wall time: 5h 6min 23s


GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor(),
                                        n_estimators=500, random_state=42),
             n_jobs=-1,
             param_grid={'base_estimator__max_depth': [None, 1, 2, 3, 4, 5, 6,
                                                       7],
                         'base_estimator__min_samples_leaf': array([11, 13, 15, 17, 19, 21])})

In [105]:
gs.best_params_

{'base_estimator__max_depth': None, 'base_estimator__min_samples_leaf': 11}

In [102]:
gs.score(X_train, y_train), gs.score(X_test, y_test)

(0.97700484918061, 0.9700940066860261)

In [103]:
gs_preds = gs.predict(X_test)
metrics.mean_absolute_error(y_test, gs_preds)

601.4636436086573